In [163]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [121]:
import pandas as pd
data = pd.read_csv("Dane/titanic.csv", decimal=",")
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
0,3,0,"Abbing, Mr. Anthony",male,42.0,0,0,C.A. 5547,7.55,NaN,S,NaN
1,3,0,"Abbott, Master. Eugene Joseph",male,13.0,0,2,C.A. 2673,20.25,NaN,S,"East Providence, RI"
2,3,0,"Abbott, Mr. Rossmore Edward",male,16.0,1,1,C.A. 2673,20.25,NaN,S,"East Providence, RI"
3,3,1,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35.0,1,1,C.A. 2673,20.25,NaN,S,"East Providence, RI"
4,3,1,"Abelseth, Miss. Karen Marie",female,16.0,0,0,348125,7.65,NaN,S,"Norway Los Angeles, CA"


In [122]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046 entries, 0 to 1045
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1046 non-null   int64  
 1   survived   1046 non-null   int64  
 2   name       1046 non-null   object 
 3   sex        1046 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1046 non-null   int64  
 6   parch      1046 non-null   int64  
 7   ticket     1046 non-null   object 
 8   fare       1045 non-null   float64
 9   cabin      272 non-null    object 
 10  embarked   1044 non-null   object 
 11  home.dest  685 non-null    object 
dtypes: float64(2), int64(4), object(6)
memory usage: 98.2+ KB


In [123]:
data.describe(include = "all")

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
count,1046.000000,1046.000000,1046,1046,1046.000000,1046.000000,1046.00000,1046,1045.000000,272,1044,685
unique,NaN,NaN,1044,2,NaN,NaN,NaN,732,NaN,174,3,348
top,NaN,NaN,"Connolly, Miss. Kate",male,NaN,NaN,NaN,CA 2144,NaN,C23 C25 C27,S,"New York, NY"
freq,NaN,NaN,2,658,NaN,NaN,NaN,8,NaN,6,782,55
mean,2.207457,0.408222,NaN,NaN,29.881135,0.502868,0.42065,NaN,36.686080,NaN,NaN,NaN
std,0.841497,0.491740,NaN,NaN,14.413500,0.912167,0.83975,NaN,55.732533,NaN,NaN,NaN
min,1.000000,0.000000,NaN,NaN,0.166700,0.000000,0.00000,NaN,0.000000,NaN,NaN,NaN
25%,1.000000,0.000000,NaN,NaN,21.000000,0.000000,0.00000,NaN,8.050000,NaN,NaN,NaN
50%,2.000000,0.000000,NaN,NaN,28.000000,0.000000,0.00000,NaN,15.750000,NaN,NaN,NaN
75%,3.000000,1.000000,NaN,NaN,39.000000,1.000000,1.00000,NaN,35.500000,NaN,NaN,NaN


In [124]:
data["pclass"]

0       3
1       3
2       3
3       3
4       3
       ..
1041    2
1042    3
1043    3
1044    3
1045    3
Name: pclass, Length: 1046, dtype: int64

In [125]:
null_counts = data.isnull().sum()
null_counts

pclass         0
survived       0
name           0
sex            0
age            0
sibsp          0
parch          0
ticket         0
fare           1
cabin        774
embarked       2
home.dest    361
dtype: int64

# pclass

In [126]:
data["pclass"] = data.pclass.astype("category")

# sex

In [127]:
from sklearn.preprocessing import OneHotEncoder

In [128]:
def encinput(df_col):
    s = pd.Series(df_col.value_counts().index)
    return list(zip(s,s.index))

In [129]:
#sex
enc = OneHotEncoder()
enc.fit(data[["sex"]])
pd.DataFrame(enc.transform(data[["sex"]]).toarray())

,0,1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,1.0,0.0
...,...,...
1041,1.0,0.0
1042,1.0,0.0
1043,0.0,1.0
1044,0.0,1.0


# fare

In [130]:
from sklearn.impute import SimpleImputer

In [131]:
imp = SimpleImputer(strategy="median")
data[["fare"]] = pd.DataFrame(imp.fit_transform(data[["fare"]]))

# cabin

In [132]:
data[["cabin"]] = pd.DataFrame(data.cabin.str[0].fillna("Empty"))
# przydałoby się zgrupować T i G z brakami
data[["cabin"]]

,cabin
0,Empty
1,Empty
2,Empty
3,Empty
4,Empty
...,...
1041,Empty
1042,Empty
1043,Empty
1044,Empty


# embarked

In [133]:
data[["embarked"]] = data[["embarked"]].fillna(pd.Series(data[["embarked"]].value_counts().index[0])) #można użyć mode

# home.dest

In [134]:
data["home.dest"].value_counts()

New York, NY                          55
London                                14
Montreal, PQ                          10
Cornwall / Akron, OH                   9
Winnipeg, MB                           8
                                      ..
Glen Ridge, NJ                         1
Stockholm, Sweden / Washington, DC     1
Dorking, Surrey, England               1
Foresvik, Norway Portland, ND          1
New York, NY / Washington, DC          1
Name: home.dest, Length: 348, dtype: int64

In [135]:
data.drop(["name", "ticket", "home.dest"], axis = 1, inplace=True)

In [136]:
X = data.drop("survived", axis=1)
y = data.survived

In [139]:
X = pd.get_dummies(X)
pd.get_dummies(X, drop_first = True) # jeżeli używalibyśmy czystej regresji logistycznej (bez regularyzacji) lepszy jest ten wariant

,age,sibsp,parch,fare,pclass_1,pclass_2,pclass_3,sex_female,sex_male,cabin_A,...,cabin_C,cabin_D,cabin_E,cabin_Empty,cabin_F,cabin_G,cabin_T,embarked_C,embarked_Q,embarked_S
0,42.0,0,0,7.5500,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,1
1,13.0,0,2,20.2500,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,1
2,16.0,1,1,20.2500,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,1
3,35.0,1,1,20.2500,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,0,1
4,16.0,0,0,7.6500,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,24.0,0,0,13.0000,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
1042,14.5,1,0,14.4542,0,0,1,1,0,0,...,0,0,0,1,0,0,0,1,0,0
1043,26.5,0,0,7.2250,0,0,1,0,1,0,...,0,0,0,1,0,0,0,1,0,0
1044,27.0,0,0,7.2250,0,0,1,0,1,0,...,0,0,0,1,0,0,0,1,0,0


In [181]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

pipe = Pipeline([("model", RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, max_depth = 10))]) 



param_grid = {'model__n_estimators': [10, 50, 100, 200],
              'model__max_leaf_nodes': [10, 20],
              'model__max_depth': [10, 20]
             }

grid = GridSearchCV(pipe, param_grid, cv=10, scoring = "accuracy", n_jobs = -1)

grid.fit(X_train, y_train)
grid.best_params_

{'model__max_depth': 10,
 'model__max_leaf_nodes': 20,
 'model__n_estimators': 200}

In [182]:
model = grid.best_estimator_
accuracy_score(y_test , model.predict(X_test))

0.780952380952381

In [183]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

pipe = Pipeline([("scaler", StandardScaler()),
                 ("model", SVC(C=1, gamma=1, probability=True))]) 



param_grid = {'model__C': [0.001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'model__gamma': ["scale", "auto"]
             }

grid = GridSearchCV(pipe, param_grid, cv=10, scoring = "accuracy", n_jobs = -1)

grid.fit(X_train, y_train)
grid.best_params_

{'model__C': 10, 'model__gamma': 'scale'}

In [184]:
model = grid.best_estimator_
accuracy_score(y_test , model.predict(X_test))

0.7857142857142857